In [31]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex 
import libsbml
import pandas as pd
import copy


In [32]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6



In [33]:
#Solver
model.solver = 'cplex'

In [34]:
#Initialize medium so as to induce Photoautotrophic conditions (i.e. no Sucrose import)
mm.read_medium_csv('../misc/photo_medium.csv', model)

#Test natin kung magwowork sa est. infinite
#Yes setting bounds to 1000 works
model_media = model.medium
model.medium

{'EX_co2(e)': 1000000,
 'EX_h2o(e)': 1000000,
 'EX_no3(e)': 1000000,
 'EX_o2(e)': 1000000,
 'EX_pi(e)': 1000000,
 'EX_so4(e)': 1000000,
 'EX_so3(e)': 1000000,
 'EX_h2s(e)': 1000000,
 'EX_nh4(e)': 1000000,
 'EX_fe2(e)': 1000000,
 'EX_fe3(e)': 1000000,
 'EX_mg2(e)': 1000000,
 'EX_photonVis(e)': 1000000}

In [35]:
# #Add constraints to model

# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")

BS_photon_import = model.reactions.PRISM_white_LED_BS
M_photon_import = model.reactions.PRISM_white_LED_M

photon_flux = model.problem.Constraint(M_photon_import.flux_expression 
                                       - BS_photon_import.flux_expression,
                                       lb = 0, ub = 1000)
model.add_cons_vars(photon_flux)


co2tex_m = model.reactions.CO2tex_M
co2tex_bs = model.reactions.CO2tex_BS

#CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
#Lower bound based  on Dark CO2 uptake rates from Ermakova et al (2020)
co2_import_cons = model.problem.Constraint(co2tex_m.flux_expression 
                                           + co2tex_bs.flux_expression, lb = 1, ub = 29)
model.add_cons_vars(co2_import_cons)

#CO2 BS intake must be limited owing to its position in the leaf tissue


# #CO2 intake needs to be mostly in the M cells. Not sure of the exact value:
# Values from Von Caemmerer (2000) in mixed C3-C4 photosynthetic plants is 1 umol CO2 m-2 s-1 per 20 CO2 m-2 s-1, which puts it at 20:1 ratio
# #try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
# #I'll assume that the ratio is at 20:1 in the meantime, meaning that there is minimal gas exchange into the BS cell that would equate to around ~1 umol CO2 m-2s-1
co2_ratio_cons = mm.set_fix_flux_ratio({co2tex_m.id:20, co2tex_bs.id:1},model)
model.add_cons_vars(co2_ratio_cons)

# #Automatic na palang nalalagay siya as constraint

# #First is the BS and M Cell constraints

In [6]:
# #What if I restrict yung import ng O2?

# #Let's try restricting O2 to the value from Lakshmanan et al. (2016) = 3.312 mmol O2 m-2 s-1
# #Actually this value is also reflected in Von Caemmerer's "Modelling C3 metabolism" (pg. 85)
# o2tex_m = model.reactions.O2tex_M
# o2tex_bs = model.reactions.O2tex_BS
# o2_import_cons = model.problem.Constraint(o2tex_bs.flux_expression, lb = -inf, ub = inf)
# model.add_cons_vars(o2_import_cons)


In [36]:
# #Add enzyme constraints
#This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
#fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
#At 0.

# #PEPC
wt_pepc = 2.4 #umol m-2 s-1
wt_mdh = 11.18 #umol m-2 s-1
wt_nadp_me = 0.14 #umol m-2 s-1
wt_ppdk = 0.31 #umol m-2 s-1
wt_CA = 15.74 #umol m-2  s-1 bar-1

#PEPC constraint (Reaction id: PPCc)
pepc_BS = model.reactions.PPCc_BS
pepc_M = model.reactions.PPCc_M

wt_pepc_cons = model.problem.Constraint(pepc_BS.flux_expression 
                                        + pepc_M.flux_expression, 
                                        lb = 0, ub = wt_pepc)
model.add_cons_vars(wt_pepc_cons)

#PPDK constraints (Reaction id: PPDKS) (note that this is found in the chloroplast?) 
#Not detected via immunolocalization but enzyme activity is detected
ppdks_BS = model.reactions.PPDKs_BS
ppdks_M = model.reactions.PPDKs_M

wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                         + ppdks_M.flux_expression, 
                                         lb = 0, ub = wt_ppdk)
model.add_cons_vars(wt_ppdks_cons)

#Malate Dehydrogenase 
mdh2c_M = model.reactions.MDH2c_M
mdh2c_BS = model.reactions.MDH2c_BS
mdh2s_M = model.reactions.MDH2s_M
mdh2s_BS = model.reactions.MDH2s_BS

wt_mdh_cons = model.problem.Constraint(mdh2c_M.flux_expression 
                                       + mdh2c_BS.flux_expression 
                                       + mdh2s_M.flux_expression 
                                       + mdh2s_BS.flux_expression, 
                                       lb= 0, ub=wt_mdh)
model.add_cons_vars(wt_mdh_cons)


#NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
#Let's see if I can force it to have a small amount of flux 
nadp_me_M = model.reactions.MDH2s_M
nadp_me_BS = model.reactions.MDH2s_BS

wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
                                         + nadp_me_BS.flux_expression,
                                         lb= wt_nadp_me, ub=wt_nadp_me)
model.add_cons_vars(wt_nadpme_cons)
#Weird, it causes the activity of RBCL to significantly vary from the actual values


#I should add constraints for Carbonic Anhydrase
#I should constrain it to 0.4 ubar, which would constitute ambient partial pressure
#
#HCO3Es
#HCO3Ec
#HCO3Em
#Flux is negative so  it needs to go backwards


hco3es_m = model.reactions.HCO3Es_M.flux_expression
hco3ec_m = model.reactions.HCO3Ec_M.flux_expression
hco3em_m = model.reactions.HCO3Em_M.flux_expression
hco3es_bs = model.reactions.HCO3Es_BS.flux_expression
hco3ec_bs = model.reactions.HCO3Ec_BS.flux_expression
hco3em_bs = model.reactions.HCO3Em_BS.flux_expression

ca_cons = model.problem.Constraint(hco3es_m + hco3ec_m + hco3em_m 
                                   + hco3es_bs + hco3ec_bs + hco3em_bs,
                                  lb = -wt_CA, ub = 0)
model.add_cons_vars(ca_cons)
#Rbcl constaints

In [38]:
#Turn off old photorespiratory reaction from the ios2164 model
model.reactions.RBPC2s_M.bounds = (0,0)
model.reactions.RBPC2s_BS.bounds = (0,0)

#Turn off H2O & Nitrate transporter in m cell
model.reactions.H2Otex_M.bounds = (0,0)
model.reactions.NO3tex_M.bounds = (0,0)


In [37]:
#Let's try to add Rubisco flux ratio

rbpc_M = model.reactions.RBPCs_M.flux_expression
rbpc_BS = model.reactions.RBPCs_BS.flux_expression
rbpo_M = model.reactions.RBPOs_M.flux_expression
rbpo_BS = model.reactions.RBPOs_BS.flux_expression

#Constraint such that it is limited to 132 umol m-2 s-1
rbcl_vcmax_cons = model.problem.Constraint(rbpc_M + rbpc_BS, lb = 0, ub= 132)

model.add_cons_vars(rbcl_vcmax_cons)

In [42]:
#Fixed ratio format (r0:value0, r1:value1)
#(value0*r1_fe - value1*r0_fe)
#Constraints for rbcl flux

rbcl_vcvo = model.problem.Constraint(3*(rbpo_M + rbpo_BS) 
                                     - 1*(rbpc_M + rbpc_BS),
                                     lb=0,ub=1000)

model.add_cons_vars(rbcl_vcvo)
#What if I simply constrained that of the M cell one to 3:1?
#This constraint is pretty good actually. 
#This allows the system to be set at a specific Vc/Vo rate while still allowing local variation 
#wherein Rubisco may act in an uncoupled fashion and may have favorable internal vc/vo rates.

In [11]:
#Modifications to medium

model_media['EX_photonVis(e)'] = 1500 #PPFD used in Ermakova et al for high light

#Set CO2 bounds to ~29 m-2 s-1
model_media['EX_co2(e)'] = 400 #mbar
model_media['EX_o2(e)'] = 200000 #mbar

#Test if heterotrophic conditions fix Biomass
#model_media['EX_sucr(e)_M'] = inf

#Nope, still doesn't produce biomass.
#How about Protons?
model_media['EX_h(e)'] = 0
#What if I constrained nitrogen uptake?
#
model_media['EX_no3(e)'] = inf
model_media['EX_nh4(e)'] = inf
model_media['EX_h2o(e)'] = inf
model.medium = model_media



In [12]:
#Turn off prism reactions except for white light


In [43]:
#Other prism reactions not turned off

with model:
    sample_fluxes = cobra.flux_analysis.pfba(model)
model.summary(fva=0.90)


Metabolite,Reaction,Flux,Range,C-Number,C-Flux
co2_e,EX_co2(e),19.1,[1; 29],1,100.00%
h2s_e,EX_h2s(e),0.02256,[0; 0.5703],0,0.00%
nh4_e,EX_nh4(e),0.5269,[0; 37.86],0,0.00%
no3_e,EX_no3(e),0.7995,[0; 37.86],0,0.00%
o2_e,EX_o2(e),23.44,[0; 2000],0,0.00%
photonVis_e,EX_photonVis(e),1311,[319.2; 5789],0,0.00%
pi_e,EX_pi(e),0.427,[0.3843; 378.4],0,0.00%
Metabolite,Reaction,Flux,Range,C-Number,C-Flux
ltlnnp_c1,DM<NNP(c)_BS,0,[-0.4151; 0],27,0.00%
ltlnnp_c0,DM<NNP(c)_M,0,[-0.4151; 0],27,0.00%


In [44]:
# This is cool, the M cell is similar to the WT rate of 83.59+-5.63 :)
print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC M', sample_fluxes['PPCc_BS'])
print('NADP-ME M', sample_fluxes['MDH2s_M'], 'NADP-ME BS', sample_fluxes['MDH2s_BS'])
print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
print('co2 consumption M', sample_fluxes[co2tex_m.id], 'co2 consumption BS', sample_fluxes[co2tex_bs.id])
print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])

rbcl M cell:  76.98560998060758 rbcl BS cell:  55.01439001939156
rbcl M cell (photorespiration) 19.711649720469758 rbcl BS cell (PR) 24.288350279529958
PEPC M 0.9066955241581572 PEPC M 0.778314881567703
NADP-ME M 0.020992294646377585 NADP-ME BS 0.11900770535362243
Biomass M:  1.206453715307578 Biomass BS 0.8629099612680513
co2 consumption M 18.190268995752973 co2 consumption BS 0.9095134497876487
Photosystem II M 98.3217792933743 PSII BS 80.4100995989437
PSI M 168.6638091520607 PSI BS 130.91877052596374


In [63]:
#Let's try restricting O2 intake again to a specific amount, let's see if PPFD responds accordingly

#Try restricting it to 10 first

with model:

    #Let's try turning off other light reactions
    for lights in model.reactions:
        if 'PRISM' in lights.id:
            if 'white' not in lights.id:
                model.reactions.get_by_id(lights.id).bounds = (0,0)
    o2tex_m = model.reactions.O2tex_M.flux_expression
    o2tex_bs = model.reactions.O2tex_BS.flux_expression
    o2_cons = model.problem.Constraint(o2tex_bs + o2tex_m, lb = -inf, ub = 3.312)
    
#     model.reactions.get_by_id('DM_o2(c)_M').objective_coefficient = 0.25
#     model.reactions.get_by_id('DM_o2(c)_BS').objective_coefficient = 0.25
   # model.reactions.get_by_id('EX_co2(e)').objective_coefficient = 0.0
    #This renders the solution as infeasible. I won't use this lol
    model.add_cons_vars(o2_cons)
    #Tignan nga natin if we change NADP-ME constraints based on Shen et al (2016)'s enzyme activity measurements
    model.add_cons_vars(model.problem.Constraint(6*nadp_me_M.flux_expression - 1*nadp_me_BS.flux_expression,lb=0, ub=0))
    
    model_media['EX_photonVis(e)'] = 2000
    model.medium = model_media
    sample_fluxes = cobra.flux_analysis.pfba(model)
    output = model.summary(fva=0.90)

In [64]:
output

In [17]:
#Note:
#Unconstrained CO2 intake leads to a saturated  net CO2 assimilation rate of around 40 umol m-2 s-1
#Better to constrain this to known uptake rates instead

In [58]:
# This is cool, the M cell is similar to the WT rate of 83.59+-5.63 :)
print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC M', sample_fluxes['PPCc_BS'])
print('NADP-ME M', sample_fluxes['MDH2s_M'], 'NADP-ME BS', sample_fluxes['MDH2s_BS'])
print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
print('co2 consumption M', sample_fluxes[co2tex_m.id], 'co2 consumption BS', sample_fluxes[co2tex_bs.id])
print('o2 consumption M', sample_fluxes['O2tex_M'], 'o2 consumption BS', sample_fluxes['O2tex_BS'])
print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])
print('PPFD M: ', sample_fluxes['PRISM_white_LED_M'], 'PPFD BS: ', sample_fluxes['PRISM_white_LED_BS'])


In [19]:
model.reactions.get_by_id('DM_o2(c)_BS')

Reaction identifier,DM_o2(c)_BS
Name,Demand reaction for oxygen
Memory address,0x07f7ca26a5520
Stoichiometry,o2_c1 --> O2 -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [20]:
model.reactions.get_by_id('DM_3-NON(s)_BS')

Reaction identifier,DM_3-NON(s)_BS
Name,Sink needed to allow (3Z)-nonenal to leave system
Memory address,0x07f7ca26a58b0
Stoichiometry,3-non_s1 --> (3Z)-nonenal -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [21]:
#Oh wtf it works, below 500 ppfd no net intake below dark respiration is observed tapos nasasaturate siya agad to  the max value
#Cooooool. Based on Weber et al (1986) the critical point is at around 500-600 umol he m-2 s-1
#Weber, J. A., Tenhunen, J. D., Gates, D. M., & Lange, O. L. (1987). Effect of photosynthetic photon flux density on carboxylation efficiency. Plant physiology, 85(1), 109-114.


In [22]:
#Weirdly enough the model doesn't produce any oxygen.

In [65]:
for pd in model.reactions:
    if 'pd' in pd.id:
        print(pd.id, sample_fluxes[pd.id])

In [24]:
#What I've done was unconstrained CO2 exchange to allow for CO2 efflux. At around 300-400 ppfd a net release of CO2 is observed from the mdoel, consistent 
#with Nobel (2020)'s  'Leaves and Fluxes'
#I need to read more on this. This is actually quite interesting


In [25]:
#Constraining O2 consumption to around 3.312 forces water to be consumed by the system, unlike when O2 is unconstrained.
#However, the same behavior when it comes to CO2 assimilation still is being produced.
#Constraining O2 consumption also forces Nitrate consumption to be constrained to realistic levels (when compared to non-constrained O2 where consumption was 10-fold )
#I suspect that this is definitely due to how PPFD flux is structured. 
#A cop-out to this would be to double PPFD per run and measuring PPFD flux in the specific reactions instead.

In [26]:
#I can probably do the benchmarking right now. The model behaves quite interestingly actually

In [27]:
#Note: Turning off other prism reactions (red, green, blue, etc) while retaining only white light reactions lowers net carbon assimilation
#I think this is in consideration on how white light is partitioned to several wavelengths, which prevents it from being fully utilized.

In [28]:
#Other artifacts as of the moment include the following:
#model doesn't produce any Oxygen. Maybe I should add a specific demand reaction for it?
#Based on FVA, the model now produces some oxygen via the demand reaction. 

In [29]:
#Model also facilitates transfer of metabolites unlike before.

In [30]:
#Questions:
#Should I restrict Nutrient flow to the Bundle Sheath Cell only or should I allow flux to both the M and BS cell?
#Apparently kasi dito only the Bundle sheath cells produce any biomass
#This is to ensure that only CO2 and Light flux are the only limiting constraints to the system.